In [ ]:
from google.colab import drive
from google.colab import auth
from google.auth import exceptions

In [ ]:
try:
  auth.authenticate_user()
  drive.mount('/content/drive')
except exceptions.RefreshError as e:
  print("Authentication error:", e)
  # Provide instructions for resolving the error, such as checking internet connectivity or refreshing credentials.

Mounted at /content/drive


In [ ]:
!pip install --upgrade gitpython

from git import Repo

import os

repo = Repo.clone_from("https://github.com/Ryma-21/Lic-PFE.git", "/content/private_repo")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00


# Dataset preprocess

In [ ]:
!pip install neattext
!pip install pdfminer
!pip install pdfminer.six

import sys
import nltk
import string
import neattext
import pdfminer
import pdfminer.high_level
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


sys.path.insert(0,"/content/private_repo/preprocessing")
from text_cleaning import remove_punctuations,remove_stopwords

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 32.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.5 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140059 sha256=935cbdf8e367cb18bf06db06bf6945eb3c756cf66bb2e6be4244372fbaec7a7a
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
Successfully built pdfminer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.8 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#spacy

In [ ]:
!pip install spacy-download
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 753.1 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

In [ ]:
nlp = spacy.load("en_core_web_lg")

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")
# Initialize the entity ruler
ruler = nlp.add_pipe("entity_ruler")
# Read patterns from the JSON file
pattern_file = "/content/drive/MyDrive/data/majors_entity_ruler_2.jsonl"
with open(pattern_file, "r") as f:
    patterns = f.readlines()
# Add each pattern to the entity ruler
for pattern_str in patterns:
    pattern_dict = json.loads(pattern_str)
    if pattern_dict.get("pattern", []):  # Check if pattern contains tokens
        ruler.add_patterns([pattern_dict])


In [ ]:
def get_majors(text):
    doc = nlp(text)
    majors = []
    for ent in doc.ents:
        if ent.label_ == "MAJOR":
            majors.append(ent.text)
    return majors


In [ ]:
# Example text
text = "I studied computer science and psychology forestry general agriculture business administration in college."

# Call the function to extract majors from the text
majors = get_majors(text)

# Print the extracted majors
print("Extracted majors:", majors)


Extracted majors: ['computer science', 'psychology', 'forestry', 'general agriculture', 'business administration']


# load jobs dataset

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/data/kaggle_recommend/jobs_with_degrees.csv")
df

,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required
0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor']
1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor']
2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor']
3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school']
4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school']
...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",
309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",
309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",
309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",


In [ ]:
df['text_non_stop'].iloc[309999]

'trade away service specialist group overviewadvisor services proud industry leading provider custodial operational trading support independent fee based investment advisors success result dedicated individuals like provide financial advisors world class personalized service unparalleled trade away service specialists provide dedicated service advisory firms build strong rapport investment advisors know first name basis provide one one service schwab known highlightsas trade away service specialist provide settlement services investment advisors maintain multiple trading relationships client accounts custodied schwab level service differentiates us competitors serves sustainable competitive advantage responsibilities support trade settlement investment managers brokerage firms ensure compliance sec finra regulations ensure proper documentation client accounts clearing broker dealer firms collaborate internal partners delivery timely accurate information response inquiries service issue

In [ ]:
get_majors("preferred bachelors degree finance economics business administration related area preferred")

['finance', 'economics', 'business administration']

In [ ]:

import re

def extract_text_after_keywords(text):
    pattern = r"(?i)((?:degree|certificate|diploma)\W+.*)"
    # Search for the pattern in the text
    match = re.search(pattern, text)
    # If a match is found, return the extracted text; otherwise, return None
    if match:
        return match.group(1)
    else:
        return None

In [ ]:
total_len = len(df)
total_len

310000

In [ ]:
# Create a new column
df['sentence_after_degree'] = None
chunksize = 100000
total_len = len(df)

# Process the DataFrame in chunks
for i in range(0, total_len, chunksize):
    # Get a chunk of data from the DataFrame
    chunk = df.iloc[i:min(i + chunksize, total_len)]

    # Apply extract sentence after keywords
    chunk['sentence_after_degree'] = chunk['text_non_stop'].apply(extract_text_after_keywords)

    # Update the original DataFrame with extracted skills
    df.loc[chunk.index, 'sentence_after_degree'] = chunk['sentence_after_degree']

    print(f"Processed chunk {i} ")


<ipython-input-243-ba35bf0e2330>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['sentence_after_degree'] = chunk['text_non_stop'].apply(extract_text_after_keywords)


Processed chunk 0 


<ipython-input-243-ba35bf0e2330>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['sentence_after_degree'] = chunk['text_non_stop'].apply(extract_text_after_keywords)


Processed chunk 100000 


<ipython-input-243-ba35bf0e2330>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['sentence_after_degree'] = chunk['text_non_stop'].apply(extract_text_after_keywords)


Processed chunk 200000 
Processed chunk 300000 


<ipython-input-243-ba35bf0e2330>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['sentence_after_degree'] = chunk['text_non_stop'].apply(extract_text_after_keywords)


In [ ]:
df

,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,sentence_after_degree
0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor'],degree experience required 3 5 years excellent...
1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor'],degree experience required 5 7 years must 5 yr...
2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor'],degree experience required 3 5 years 5 7 year ...
3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school'],diploma ged required must successfully pass ba...
4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school'],certificate college technical school accountin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",,None
309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",,None
309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",,None
309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",,None


In [ ]:
# Replace None values with an empty string
df['sentence_after_degree'] = df['sentence_after_degree'].fillna(' ')

In [ ]:
df

,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,sentence_after_degree
0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor'],degree experience required 3 5 years excellent...
1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor'],degree experience required 5 7 years must 5 yr...
2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor'],degree experience required 3 5 years 5 7 year ...
3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school'],diploma ged required must successfully pass ba...
4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school'],certificate college technical school accountin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",,
309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",,
309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",,
309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",,


In [ ]:
# Create a new column
df['major_required'] = None
chunksize = 10000
total_len = len(df)

# Process the DataFrame in chunks
for i in range(0, total_len, chunksize):
    # Get a chunk of data from the DataFrame
    chunk = df.iloc[i:min(i + chunksize, total_len)]

    # Apply get majors after keywords
    chunk['major_required'] = chunk['sentence_after_degree'].apply(get_majors)

    # Update the original DataFrame with extracted majors
    df.loc[chunk.index, 'major_required'] = chunk['major_required']

    print(f"Processed chunk {i} ")

<ipython-input-249-d180a0c557d0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['major_required'] = chunk['sentence_after_degree'].apply(get_majors)


Processed chunk 0 


<ipython-input-249-d180a0c557d0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['major_required'] = chunk['sentence_after_degree'].apply(get_majors)


Processed chunk 10000 


<ipython-input-249-d180a0c557d0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['major_required'] = chunk['sentence_after_degree'].apply(get_majors)


...

<ipython-input-249-d180a0c557d0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['major_required'] = chunk['sentence_after_degree'].apply(get_majors)


Processed chunk 280000 


<ipython-input-249-d180a0c557d0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['major_required'] = chunk['sentence_after_degree'].apply(get_majors)


Processed chunk 290000 
Processed chunk 300000 


<ipython-input-249-d180a0c557d0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['major_required'] = chunk['sentence_after_degree'].apply(get_majors)


In [ ]:
df.drop(columns=['combined_text'])

,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,sentence_after_degree,major_required
0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor'],degree experience required 3 5 years excellent...,
1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor'],degree experience required 5 7 years must 5 yr...,
2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor'],degree experience required 3 5 years 5 7 year ...,
3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school'],diploma ged required must successfully pass ba...,
4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school'],certificate college technical school accountin...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",,,
309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",,,
309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",,,
309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",,,


In [ ]:
df["major_required"].value_counts()

major_required
                                                                                254318
[finance, accounting]                                                             5979
[accounting]                                                                      5815
[communications]                                                                  5155
[history]                                                                         4469
                                                                                 ...  
[economics, nursing, communications]                                                 1
[mathematics, elementary education]                                                  1
[international business, geography, accounting]                                      1
[biology, chemistry, communications, microbiology]                                   1
[mathematics, physics, chemistry, communications, history, computer science]         1
Name: count, Length: 1996, d

In [ ]:
df['major_required'] = df['major_required'].apply(lambda x: list(set(x)))

In [ ]:
df['major_required'] = df['major_required'].apply(lambda x: '' if len(x) == 0 else x)

In [ ]:
df.at[3, 'major_required'] = ''

In [ ]:
major_counts = df['major_required'].value_counts()
major_counts

major_required
                                                                                254318
[finance, accounting]                                                             5979
[accounting]                                                                      5815
[communications]                                                                  5155
[history]                                                                         4469
                                                                                 ...  
[economics, nursing, communications]                                                 1
[mathematics, elementary education]                                                  1
[international business, geography, accounting]                                      1
[biology, chemistry, communications, microbiology]                                   1
[mathematics, physics, chemistry, communications, history, computer science]         1
Name: count, Length: 1996, d

In [ ]:
df.to_csv('/content/drive/MyDrive/data/kaggle_recommend/jobs_with_education.csv')

In [ ]:
new_df = pd.read_csv('/content/drive/MyDrive/data/kaggle_recommend/jobs_with_education.csv')
new_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,sentence_after_degree,major_required
0,0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor'],degree experience required 3 5 years excellent...,NaN
1,1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor'],degree experience required 5 7 years must 5 yr...,NaN
2,2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor'],degree experience required 3 5 years 5 7 year ...,NaN
3,3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school'],diploma ged required must successfully pass ba...,NaN
4,4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school'],certificate college technical school accountin...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",,,NaN
309996,309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",,,NaN
309997,309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",,,NaN
309998,309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",,,NaN


In [ ]:
# remove nan values
new_df['major_required'].fillna('', inplace=True)
new_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,sentence_after_degree,major_required
0,0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor'],degree experience required 3 5 years excellent...,
1,1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor'],degree experience required 5 7 years must 5 yr...,
2,2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor'],degree experience required 3 5 years 5 7 year ...,
3,3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school'],diploma ged required must successfully pass ba...,
4,4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school'],certificate college technical school accountin...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",,,
309996,309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",,,
309997,309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",,,
309998,309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",,,


In [ ]:
df.to_csv('/content/drive/MyDrive/data/kaggle_recommend/jobs_with_education_1.csv')

In [ ]:
new_df['major_required'].value_counts()

major_required
                                                                                            254318
['finance', 'accounting']                                                                     5979
['accounting']                                                                                5815
['communications']                                                                            5155
['history']                                                                                   4469
                                                                                             ...  
['economics', 'nursing', 'communications']                                                       1
['mathematics', 'elementary education']                                                          1
['international business', 'geography', 'accounting']                                            1
['biology', 'chemistry', 'communications', 'microbiology']                                    

In [ ]:
# Assuming your DataFrame is called df

# Filter out rows where 'major_required' column is not empty
non_empty_majors = new_df[new_df['major_required'].apply(lambda x: len(x) > 0)]

# Display the filtered DataFrame
non_empty_majors


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,sentence_after_degree,major_required
12,12,12,12,458088,Denver,CO,US,sharepoint developer,sharepoint developer,sharepoint developer veriant is currently seek...,sharepoint developer veriant currently seeking...,"['sharepoint', 'seeking', 'downtown', 'direct ...",['master'],degree bs computer science equivalent desired ...,['computer science']
17,17,17,17,458103,Plymouth,MN,US,quality engineer mechanical,quality engineer mechanical,quality engineer mechanical the quality engi...,quality engineer mechanical quality engineer m...,"['strategy', 'engineering', 'product', 'lifecy...","['bachelor', 'master']",degree mechanical engineering equivalent maste...,['mechanical engineering']
18,18,18,18,458124,Indianapolis,IN,US,provider contracting representative,provider contracting representative,provider contracting representative ourclient ...,provider contracting representative ourclient ...,"['savings', 'clients', 'public', 'customer ser...",['high school'],diploma ged required two years customer servic...,['communications']
19,19,19,19,458128,Fredericksburg,VA,US,ecologist ii wetland delineator,ecologist ii wetland delineator,ecologist ii wetland delineator williamsburg e...,ecologist ii wetland delineator williamsburg e...,"['resumes', 'design', 'support services', 'pub...",,degree environmental science ecology environme...,"['environmental science', 'ecology']"
20,20,20,20,458133,Phoenix,AZ,US,sql database administrator,sql database administrator,sql database administrator city of phoenix sql...,sql database administrator city phoenix sql da...,"['support', 'sql', 'database', 'administration...",['bachelor'],degree computer science computer information s...,"['business administration', 'mathematics', 'co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309968,309968,309968,179968,944615,Calabasas,CA,US,manager business analytics,manager business analytics,manager business analytics we are seeking a m...,manager business analytics seeking manager bus...,"['business analytics', 'seeking', 'business an...",,degree economics finance quantitative science ...,"['economics', 'finance', 'computer science']"
309980,309980,309980,179980,946866,Portland,OR,US,cl portfolio manager,cl portfolio manager,cl portfolio manager responsible for leading a...,cl portfolio manager responsible leading proje...,"['project', 'development', 'technology', 'busi...",['bachelor'],degree computer science computer science infor...,"['architecture', 'computer science']"
309981,309981,309981,179981,947111,Walnut Creek,CA,US,student trainee,student trainee,student trainee travel n a brown and caldwell ...,student trainee travel n brown caldwell provid...,"['travel', 'municipalities', 'private', 'gover...",['bachelor'],degree civil environmental mechanical engineer...,['mechanical engineering']
309991,309991,309991,179991,948242,Corpus Christi,TX,US,rn medical surgical unit,rn medical surgical unit,rn medical surgical unit committed to the care...,rn medical surgical unit committed care improv...,"['medical surgical', 'improvement', 'life', 'c...",['associate'],degree diploma nursing bsnlicensure certificat...,['nursing']


# majors from job desc

In [ ]:
df['major_required'] = df['text_non_stop'].apply(get_majors)

In [ ]:
df

,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,major_required
0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor'],[]
1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor'],[]
2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor'],"[accounting, finance]"
3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school'],[history]
4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school'],[accounting]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",,[]
309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",,[]
309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",,[]
309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",,"[communications, communications, communication..."


# preprocess dataset

In [ ]:
 # replace [] with empty space
df['major_required'] = df['major_required'].apply(lambda x: ' ' if len(x) == 0 else x)
df


,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,major_required
0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor'],
1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor'],
2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor'],"[accounting, finance]"
3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school'],[history]
4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school'],[accounting]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",,
309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",,
309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",,
309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",,"[communications, communications, communication..."


In [ ]:
df['major_required'].value_counts()

major_required
                                                                                                                                          188952
[communications]                                                                                                                           12345
[history]                                                                                                                                  11311
[accounting]                                                                                                                                6224
[finance]                                                                                                                                   4902
                                                                                                                                           ...  
[microbiology, accounting, microbiology, chemistry]                                                                

In [ ]:
# Remove duplicates from lists in the 'major_required' column
df['major_required'] = df['major_required'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)

# Print the DataFrame to verify the removal of duplicates
df


,Unnamed: 0.1,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills,degree_required,major_required
0,0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca...",['bachelor'],
1,1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se...",['bachelor'],
2,2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic...",['bachelor'],"[finance, accounting]"
3,3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ...",['high school'],[history]
4,4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',...",['high school'],[accounting]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti...",,
309996,309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '...",,
309997,309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ...",,
309998,309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services...",,[communications]


In [ ]:
df['major_required'].value_counts()

major_required
                                                                                                 188952
[communications]                                                                                  16908
[finance, accounting]                                                                             14680
[history]                                                                                         13104
[accounting]                                                                                      11954
                                                                                                  ...  
[computer engineering, architecture, computer science, aerospace engineering, communications]         1
[geography, chemistry, chemical engineering, history]                                                 1
[finance, computer science, accounting, physics, history]                                             1
[finance, general business, public administration

In [ ]:
# Count the number of rows where degrees are required and the value is not an empty space
non_empty_degrees_count = len(df[df['degree_required'].str.strip() != ''])

# Print the count
print("Number of rows with non-empty degrees required:", non_empty_degrees_count)


Number of rows with non-empty degrees required: 158778
